In [ ]:
from google.colab import files
import caffe
import numpy as np
import cv2
import os
from pylab import imread,subplot,imshow,show
import matplotlib.pyplot as plt

MEAN_COLOR = np.array([92.8207477031, 95.2757037428, 104.877445883]).reshape((1, 3, 1, 1)) # BGR

deploy = '../CrowdEstimation-model/CSRNet/CSRNet-caffe/models/deploy.prototxt'
caffe_model = '/content/drive/ml/CSRNet-mx/models/shanghaib.caffemodel'

nn = caffe.Net(deploy, caffe_model, caffe.TEST) 

def predict(imgs):
    nn.blobs["data"].reshape(*(imgs.shape))
    nn.blobs["data"].data[...] = imgs
    nn.forward()
    ps = nn.blobs["estdmap"].data.sum((1,2,3))
    return ps 

if __name__ == '__main__':
    import os
    def is_image_file(fname):
        ext = os.path.splitext(fname)[-1].lower()
        return ext == '.jpg'
    data_path = '/content/drive/ml/CSRNet-mx/data/shanghaitech/part_B_final/test_data/'
    image_names = list(filter(is_image_file, os.listdir(os.path.join(data_path, 'images'))))
    image_paths = [os.path.join(data_path, 'images', e) for e in image_names]

    fout = open('predict.txt', 'w')
    num_images = len(image_names)
    for i, fname in enumerate(image_paths):
        img_name = image_names[i]
        im = cv2.imread(fname)
        rows, cols, ts = im.shape
        imgs = im.transpose((2,0,1)).reshape((1, 3, rows, cols)) - MEAN_COLOR
        plt.imshow(im)        
        plt.savefig(fname,format='png',bbox_inches=None, pad_inches=0.1,frameon=None)
        plt.show()
        ps = predict(imgs)
        p = ps[0]
        print ('{}/{}: {} {}'.format(i + 1, num_images, img_name, p))
        fout.write('{} {}\n'.format(img_name, p))

        i += 1